# Connect4-Zero Training

AlphaZero-style Connect 4. **Setup:** Runtime > Change runtime type > GPU

In [ ]:
# 1. Setup (run once)
!rm -rf /content/connect4-zero
!git clone https://github.com/mindswim/connect4-zero.git /content/connect4-zero
!pip install -e /content/connect4-zero -q

import sys
sys.path.insert(0, '/content/connect4-zero/src')

import torch
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

In [ ]:
# 2. Train (turbo config ~30 sec/iteration)
%cd /content/connect4-zero
!c4z train --config configs/turbo.yaml --iterations 100

In [ ]:
# 3. Test vs random (run after training OR after runtime restart)
import sys
sys.path.insert(0, '/content/connect4-zero/src')

from connect4zero.net import load_checkpoint
from connect4zero.eval import Arena
from connect4zero.utils import get_device

device = get_device()
model, _ = load_checkpoint('/content/connect4-zero/checkpoints/best.pt', device)

arena = Arena(num_simulations=50, device=device)
result = arena.evaluate_vs_random(model, num_games=50)
print(f"\nWin rate vs random: {result.win_rate*100:.1f}%")
print(f"W:{result.wins} L:{result.losses} D:{result.draws}")

In [ ]:
# 4. Save to Google Drive
from google.colab import drive
drive.mount('/content/drive')
!mkdir -p /content/drive/MyDrive/connect4-zero
!cp /content/connect4-zero/checkpoints/best.pt /content/drive/MyDrive/connect4-zero/best.pt
print("Saved to Google Drive!")

In [ ]:
# 5. Download to computer
from google.colab import files
files.download('/content/connect4-zero/checkpoints/best.pt')

## Optional: Train longer with stronger config
```python
!c4z train --resume /content/connect4-zero/checkpoints/best.pt --config configs/v1.yaml --iterations 200
```